# Eksperimenti na diedrski grupi
- P(gf z P(0) ~ uniform) ~= 20%? 

## 2. Verjetnost uspešne konvergence
Zanima nas $P( \mathcal L \left(\lim \limits_{t \to \infty}\phi(t) \right) )$

Za n =1 , 2, ..., 10 za vsak D_2n samplamo n_samples=5000 različnih parametrov in z njimi rešujemo ode

In [1]:
import numpy as np
from  gofi.ode.dihedral.plot import get_characters, get_matrices


In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from gofi.ode.dihedral.gradient_flow import GradientFlow
from gofi.ode.dihedral.loss import IrreducibilityLoss, UnitaryLoss, RelationLoss

plot_size=500

probs=[]

for n in range(3, 14):
    filename = f'/home/urh/Documents/fmf/magistrska-naloga/gofi/tests/Dn_2dim_sample_20_10000n={n}.npy'
    solutions = np.load(filename, allow_pickle=True)

    gf = GradientFlow(2, 
    IrreducibilityLoss(n) +
    UnitaryLoss() +
    RelationLoss(n), 10)

    

    n_diverges=0



    fig, axs = plt.subplots(1)
    
    plotted = 0
    

    for solution in solutions:

        # get final matrices
        R, S = get_matrices(solution.y, -1, 2)
        loss = gf.loss(R, S)
        
        charR, charS = get_characters(solution.y, 2)
        if loss > 0.1:
           
            n_diverges += 1
           # color = (56/255, 101/255, 75/255, 0.27)
            color = (232/255, 148/255, 40/255, 0.35)
            zorder = 3
        else:
            color = color=(8/255, 96/255, 204/255)
            zorder = 4
        
        if plotted < plot_size:
            # plot curve
            axs.plot(charR, charS, color=color, zorder=zorder)
            # plot initial point
            axs.plot(charR[0], charS[0], '.k', label='Initial point', markersize=3, zorder=5)
            plotted += 1
        

    axs.set_xlabel('tr(R)')
    axs.set_ylabel('tr(S)')
    #axs.legend()
    
    P = (len(solutions) - n_diverges)/len(solutions)
    probs.append(P)
    fig.suptitle(f'Trajektorije različnih začetnih vrednosti modela  $\\rho  \\colon D_{{{n}}} \\to \\mathbb{{R}}^{{2 \\times 2}}$')
   # axs.set_title(r'$P(\mathcal{L}(\lim _{t \to \infty } \phi(t) ) = 0 ) = $' + str(P))
    plt.savefig(f"trajektorije_D_{n}.pdf")
    plt.show()

print(probs)


FileNotFoundError: [Errno 2] No such file or directory: '/home/urh/Documents/fmf/magistrska-naloga/gofi/tests/Dn_2dim_sample_20_10000n=3.npy'

In [ ]:
for i, p in enumerate(probs):
    n = i + 3
    #perc = p * 100
    fancy = round(p, 2) 
    print(f"$D_{{{n}}}$ & ${fancy}$\\\\")

## 1. Ali matrike z istim karakterjem konvergirajo k istim vrednostim?
**NE!!!!**
### Plan dela
Za različne vrednosti karakterjev vzorčimo M naključnih matrik $R,S$ z danima karakterjema $charR, charS \in [-N, N]$. 

In [ ]:
from  gofi.ode.dihedral.plot import get_characters, get_matrices
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.subplot(111)

def plot_sample(sample, char=None):
    init_points = sample['init_points']
    solutions = sample['solutions']
    for i in range(len(init_points)):
        # calculate loss
        R, S = get_matrices(solutions[i].y, -1, 2)
        R0, S0 = get_matrices(solutions[i].y, 0, 2)
        l = loss(R, S)
        

        # get characters of i-th trajectory
        charR, charS = get_characters(solutions[i].y, 2)
        # plot trajectory 
        plt.plot(charR, charS, label=f"#{i}, loss={l}, detR={np.linalg.det(R0)}, detS={np.linalg.det(S0)}")
        # plot initial point
        plt.plot(charR[0], charS[0], 'xb', markersize=4) 
        # plot final point
        plt.plot(charR[-1], charS[-1], 'xr', markersize=4) 
    plt.xlabel('trace(R)')
    plt.ylabel('trace(S)')
    plt.title(f"Trajectories of different matrices with charR={charR[0]}, charS={charS[0]}")
    plt.legend()

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

plot_sample(samples[0])

for i, (R,S) in enumerate(samples[0]['init_points']): 
    pass
    #print(f"Sample {i}: R=\n{R}, S=\n{S}")



In [ ]:
from tqdm import tqdm
trR = -1 
antidiagR = -1 

trS = 0.4
antidiagS = 0.6



fig = plt.figure()
ax = plt.subplot(111)
n_samples = 50
for i in tqdm(range(n_samples), total=n_samples):
    a = np.random.rand()*2 - 1
    b = np.random.rand()*2 - 1
    R0 = np.array([[a, b],[antidiagR - b, trR - a]]) 
    a = np.random.rand()*2 - 1
    b = np.random.rand()*2 - 1
    S0 = np.array([[a, b],[antidiagS - b, trS - a]]) 
    # run 
    solution = gf.solve(R0, S0, t_max=5)
    
    # get loss
    R, S = get_matrices(solution.y, -1, 2)
    l = loss(R, S)
    # get characters
    charR, charS = get_characters(solution.y, 2)
    # plot trajectory 
    plt.plot(charR, charS, label=f"#{i}, loss={l}, detR={np.linalg.det(R0)}, detS={np.linalg.det(S0)}")
    # plot initial point
    plt.plot(charR[0], charS[0], 'xb', markersize=4) 
    # plot final point
    plt.plot(charR[-1], charS[-1], 'xr', markersize=4) 
plt.xlabel('trace(R)')
plt.ylabel('trace(S)')
plt.title(f"Trajectories of different matrices with charR={charR[0]}, charS={charS[0]}")
plt.legend()

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()
    